# Experiment Workbench

In [1]:
import os, sys, math, datetime
import pathlib
import numpy as np
import random
from matplotlib import pyplot as plt
import PIL
import PIL.Image

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from dotenv import load_dotenv
import boto3
import wandb
from wandb.keras import WandbCallback
import mlflow


In [2]:

print("Tensorflow version " + tf.__version__)
AUTOTUNE = tf.data.AUTOTUNE

# Confirm that TensorFlow can access GPU
device_name = tf.test.gpu_device_name()
if not device_name:
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Tensorflow version 2.10.0
Found GPU at: /device:GPU:0


In [3]:
# Start a Tensorboard session
%load_ext tensorboard

In [11]:
os.environ['WANDB_NOTEBOOK_NAME'] = 'Experiment Workbench'



BATCH_SIZE = 128
EPOCHS = 5
LOGGING_STEPS = 10
LR = 0.001

PROJECT = "Tiny CNN"
MODELNAME = "Simple_Net"
EXPERIMENT = "MNIST"
RUN_NAME = "Run_8"

logdir = os.path.join("logs", MODELNAME, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

# Preparing the data

In [4]:
# Load some data
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
assert x_train.shape == (50000, 32, 32, 3)
assert x_test.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 1)
assert y_test.shape == (10000, 1)

x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
#y_train = keras.utils.to_categorical(y_train, 10)
#y_test = keras.utils.to_categorical(y_test, 10)

input_shape = x_train.shape[1:]
classes = 10
labels = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

In [6]:
def data_preprocessing(image, label):
    # this function can be extended if any pre-processing or augmentation is needed
    return (image, label)

In [7]:

# Create a tf.data pipeline of augmented images (and their labels)
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
#train_dataset = train_dataset.batch(BATCH_SIZE).map(lambda x, y: (data_augmentation(x), y))
train_dataset = train_dataset.batch(BATCH_SIZE).map(data_preprocessing).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
#train_dataset = train_dataset.batch(BATCH_SIZE).map(lambda x, y: (data_augmentation(x), y))
val_dataset = val_dataset.batch(BATCH_SIZE).map(data_preprocessing).prefetch(tf.data.AUTOTUNE)


# Define the model

In [8]:
def create_model(input_shape, classes):
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=input_shape),
    #tf.keras.layers.Rescaling(1.0 / 255),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(classes, activation='softmax')
  ])

In [17]:
def train_model():

        # solve issue from: https://github.com/wandb/wandb/issues/3536
        if len(wandb.patched["tensorboard"]) > 0:
                wandb.tensorboard.unpatch()
                
        # Configure Tensorboard root log directory to read the debugging information
        
        # wandb.tensorboard.patch(root_logdir="./logs/debug")
        # wandb.tensorboard.patch(root_logdir="wandb.run.dir")
        
        wandb.init(
                # Set the project where this run will be logged
                project=PROJECT, 
                # Track hyperparameters and run metadata
                #config={
                #"learning_rate": LR,
                #"epochs": EPOCHS,
                #},
                sync_tensorboard=True
                )


        config = wandb.config
        # Specify the configuration variables
        config.dropout = 0.2
        #config.hidden_layer_size = 128
        #config.layer_1_size  = 16
        #config.layer_2_size = 32
        config.learn_rate = LR
        #config.decay = 1e-6
        #config.momentum = 0.9
        config.epochs = EPOCHS
        config.classes = classes

        # enable Tensorflow Debugging
        #tf.debugging.experimental.enable_dump_debug_info("./logs/debug", 
        #        tensor_debug_mode="FULL_HEALTH", circular_buffer_size=-1)

        
        model = create_model(input_shape, classes)
        model.compile(optimizer='adam',
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])

        logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir= wandb.run.dir, histogram_freq=1, profile_batch="10, 20")
        #tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir= logdir, histogram_freq=1)
        wandb_callback = WandbCallback(input_type="image", labels=labels)

        early_stopping = EarlyStopping(monitor="val_accuracy", patience=3)

        checkpoint = ModelCheckpoint("my_tiny_model", save_weights_only=True)
        history = model.fit(train_dataset,
                epochs=config.epochs, 
                validation_data=val_dataset, 
                callbacks=[tensorboard_callback, wandb_callback, checkpoint, early_stopping])

        wandb.finish()
        return history, model


In [18]:
history, model = train_model()

Epoch 1/5
388/391 [============================>.] - ETA: 0s - loss: 1.9945 - accuracy: 0.2949

wandb: WARNING When using several event log directories, please call `wandb.tensorboard.patch(root_logdir="...")` before `wandb.init`


2/2 [==============================] - 0s 5ms/step
INFO:tensorflow:Assets written to: i:\tinyml\tiny_cnn\wandb\run-20221016_221100-230352ul\files\model-best\assets


wandb: Adding directory to artifact (i:\tinyml\tiny_cnn\wandb\run-20221016_221100-230352ul\files\model-best)... Done. 0.1s


391/391 [==============================] - 11s 27ms/step - loss: 1.9934 - accuracy: 0.2953 - val_loss: 1.7843 - val_accuracy: 0.3692
Epoch 2/5
2/2 [==============================] - 0s 4ms/step - loss: 1.7951 - accuracy: 0.35
INFO:tensorflow:Assets written to: i:\tinyml\tiny_cnn\wandb\run-20221016_221100-230352ul\files\model-best\assets


...

wandb: Adding directory to artifact (i:\tinyml\tiny_cnn\wandb\run-20221016_221100-230352ul\files\model-best)... Done. 0.1s


391/391 [==============================] - 8s 19ms/step - loss: 1.7054 - accuracy: 0.3923 - val_loss: 1.6378 - val_accuracy: 0.4167
Epoch 5/5
2/2 [==============================] - 0s 5ms/step - loss: 1.6790 - accuracy: 0.40
INFO:tensorflow:Assets written to: i:\tinyml\tiny_cnn\wandb\run-20221016_221100-230352ul\files\model-best\assets


wandb: Adding directory to artifact (i:\tinyml\tiny_cnn\wandb\run-20221016_221100-230352ul\files\model-best)... Done. 0.1s


391/391 [==============================] - 7s 18ms/step - loss: 1.6789 - accuracy: 0.4028 - val_loss: 1.6278 - val_accuracy: 0.4185


accuracy,▁▅▇▇█
epoch,▁▃▅▆█
loss,█▄▂▂▁
val_accuracy,▁▅███
val_loss,█▅▂▁▁
accuracy,0.4028
best_epoch,4
best_val_loss,1.62782
epoch,4
loss,1.67891
val_accuracy,0.4185


In [7]:
def run_multiple_training_runs(epochs, lrs):
    for epoch in epochs:
        for lr in lrs:
            run_training_run(epoch, lr)

# Try different values for the learning rate
epochs = [100, 120, 140]
lrs = [0.1, 0.01, 0.001, 0.0001]
run_multiple_training_runs(epochs, lrs)

Training for 100 epochs with learning rate 0.1


lr: 0.1
epoch=2, acc=0.3239496078287173, loss=0.7840237407873047
epoch=3, acc=0.4886583526039714, loss=0.5382489089033355
epoch=4, acc=0.5946108236677674, loss=0.3062215871379621
epoch=5, acc=0.842835687320363, loss=0.15326649102771978
epoch=6, acc=0.841041689897303, loss=0.2773970852560576
epoch=7, acc=0.7651799556122217, loss=0.13211731915830313
epoch=8, acc=0.8289271781530685, loss=0.14014753902136384
epoch=9, acc=0.87325792818547, loss=0.20998891682843918
epoch=10, acc=0.8865158378162138, loss=0.18009344870749755
epoch=11, acc=0.8765156790767422, loss=0.18896313900651218
epoch=12, acc=0.846993952368912, loss=0.12943821152351473
epoch=13, acc=0.8658211826520751, loss=0.12168794547255812
epoch=14, acc=0.8482459514679072, loss=0.12455625352218093
epoch=15, acc=0.8344692612820243, loss=0.15507962690026417
epoch=16, acc=0.8574632130602637, loss=0.13816451456866294
epoch=17, acc=0.8773946264610468, loss=0.15946654792443482
epoch=18, acc=0.8607377277060754, loss=0.11342147994528552
epoch=

acc,▁▄▇██▇█▇▇███████████████████████████████
loss,█▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
acc,0.8835
loss,0.11537


Training for 100 epochs with learning rate 0.01


lr: 0.01
epoch=2, acc=0.29526289920347026, loss=0.8341154540113698
epoch=3, acc=0.6755546341923118, loss=0.5446967698847112
epoch=4, acc=0.5687387351564727, loss=0.2501680595162399
epoch=5, acc=0.6479683009706949, loss=0.2944196200799194
epoch=6, acc=0.7269402229238164, loss=0.17455103262257554
epoch=7, acc=0.7479877366112375, loss=0.21084122721524995
epoch=8, acc=0.8105065539378978, loss=0.22067878846980743
epoch=9, acc=0.8325027806706508, loss=0.18853837829622858
epoch=10, acc=0.8317305553182309, loss=0.2465066785958966
epoch=11, acc=0.8257012452893249, loss=0.22432187681031196
epoch=12, acc=0.7759824041792263, loss=0.20358038797206052
epoch=13, acc=0.8190111966715984, loss=0.1704200023442612
epoch=14, acc=0.8107537732866739, loss=0.20980087170856698
epoch=15, acc=0.7875499701821629, loss=0.2168608641192949
epoch=16, acc=0.8299655402948874, loss=0.16539888354403562
epoch=17, acc=0.8169154295673712, loss=0.18576791821041078
epoch=18, acc=0.8437036700100868, loss=0.18384802230941177
ep

...

lr: 0.001
epoch=2, acc=0.28938460854729037, loss=0.44886805705386723
epoch=3, acc=0.7354552547802814, loss=0.37732757982077375
epoch=4, acc=0.8182746686145462, loss=0.29193159965786614
epoch=5, acc=0.921011901108274, loss=0.20323057626193872
epoch=6, acc=0.8793574266838373, loss=0.15046385331501033
epoch=7, acc=0.9561086989099777, loss=0.07801182720204888
epoch=8, acc=0.9412237153154888, loss=0.11587035446983167
epoch=9, acc=0.8771294910264921, loss=0.06928205376738089
epoch=10, acc=0.9299387322084643, loss=0.09097187041596023
epoch=11, acc=0.9327730473108038, loss=0.06104266965248688
epoch=12, acc=0.9740567740814317, loss=0.021010613986532728
epoch=13, acc=0.9698100537388907, loss=0.021840935603716978
epoch=14, acc=0.9807370033917441, loss=0.07851995571058865
epoch=15, acc=0.9318311752615941, loss=0.026238196893773595
epoch=16, acc=0.9382343313318273, loss=0.026656176833814087
epoch=17, acc=0.9669034443518513, loss=0.03056537974425458
epoch=18, acc=0.9701615435464694, loss=0.060071903

acc,▁▆▅█▇▇▇▇██▇███▇█████████████████████████
loss,█▅▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
acc,0.97725
loss,0.02112


Training for 140 epochs with learning rate 0.0001


lr: 0.0001
epoch=2, acc=0.18357633028736153, loss=0.6586663198538287
epoch=3, acc=0.49256123192114637, loss=0.5735444214888641
epoch=4, acc=0.7202133129709583, loss=0.31401781583024824
epoch=5, acc=0.7668549417315831, loss=0.2491586578531868
epoch=6, acc=0.8232129491956386, loss=0.207909622275774
epoch=7, acc=0.7993729153478095, loss=0.2713993996607026
epoch=8, acc=0.7851404040594507, loss=0.19814035733124974
epoch=9, acc=0.7911624848660299, loss=0.24094754450373218
epoch=10, acc=0.8483035731205382, loss=0.15124632617492453
epoch=11, acc=0.8337180273877249, loss=0.19795850968607978
epoch=12, acc=0.8009205424349133, loss=0.1479611882852023
epoch=13, acc=0.8243392898809849, loss=0.1583758122698895
epoch=14, acc=0.8094129577464102, loss=0.14904193247462152
epoch=15, acc=0.7972956449856028, loss=0.16052079265257607
epoch=16, acc=0.8507626792257523, loss=0.18599703243337892
epoch=17, acc=0.8618162751003089, loss=0.1866355702311686
epoch=18, acc=0.842344400961329, loss=0.16660512081310017
ep

acc,▁▆▇▇▇███▇███████████████████████████████
loss,█▃▃▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
acc,0.86007
loss,0.1446
